# **Scraping**
## imports

In [91]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Extraccion de datos
- equipos -> [[equipos_locales], [equipos_visitantes]]
- cuotas -> [[cuota_local, cuota_empate, cuota_visitante]] 
- fecha_hora -> [[fecha], [hora]]

In [92]:
class Scraping:
    def __init__(self, urls) -> None:
        for url in urls:
            print(f"analizando >>{url}<<")
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'lxml')
            self.container = soup.find('div', class_='fragment expander coupon-for-type')

            duplicade_links = [elem.get("href") for elem in self.container.find_all('a', title="Número de mercados")]
            self.links_per_match = [] 
    
            for item in duplicade_links:
                if item not in self.links_per_match:
                    self.links_per_match.append(item)

    def equipos(self):
        local_vist = []
        equipos = [elem.text for elem in self.container.find_all('span', class_='seln-name')]
        locales = equipos[::2]
        visitantes = equipos[1::2]
        for i in range(len(locales)):
            match = [locales[i], visitantes[i]]
            local_vist.append(match)
        return local_vist
    
    def cuotas(self):
        puntos = [elem.text for elem in self.container.find_all('span', class_='price dec')]
        cuotas = [puntos[i:i+3] for i in range(0, len(puntos), 3)]
        return cuotas
    
    def hora_fecha(self):
        meses = {'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06',
             'Jul': '07', 'Ago': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'}
        horarios = []
        datos = pd.DataFrame(columns=['hora', 'fecha'])
        eventos = self.container.find_all('div', {'class': 'ev'})
        for evento in eventos:
            hora_element = evento.find('span', {'class': 'time'})
            fecha_element = evento.find('span', {'class': 'date'})

            if hora_element and fecha_element:
                hora = hora_element.text
                fecha = fecha_element.text
                fecha = fecha + ' 2023'
                fecha = fecha.replace(fecha.split()[1], meses[fecha.split()[1]])
                fecha_hora = fecha + ' ' + hora
                fecha_hora = datetime.strptime(fecha_hora, '%d %m %Y %H:%M')
                fecha_hora = datetime.strftime(fecha_hora, '%Y-%m-%d %H:%M')
                horarios.append(str(fecha_hora).split(' '))
        return horarios
    
    def dobles_oportunidades(self):
        dobles_oportunidades = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('ul', class_='default mkt_content limited mkt-sort-DBLC ev-sort-MT')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                dobles_oportunidades.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene dobles_oportunidades")
                dobles_oportunidades.append(["N/A","N/A", "N/A"])
        return dobles_oportunidades
    
    def ambos_anotan(self):
        si_no = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('table', class_='horizontal mkt_content mkt-sort-BTSC')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                si_no.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene ambos_anotan")
                si_no.append(["N/A","N/A"])
        return si_no




## creacion del excel

In [95]:
class Create_Excel(Scraping):
    def __init__(self, urls) -> None:
        super().__init__(urls)
        
        equipos = self.equipos()
        cuotas = self.cuotas()
        hora_fecha = self.hora_fecha()
        dobles_oportunidades = self.dobles_oportunidades()
        si_no = self.ambos_anotan()
        total_data = zip(equipos, cuotas, hora_fecha,
                        dobles_oportunidades, si_no)
        COLUMNS = ['local', 'vist', 'cuo_local', 'cuo_emp', 'cuo_vist', 'fecha', 'hora',
                   'local_emp', 'local_vist', 'vist_emp', 'si_anotan', 'no_anotan']
        self.Excel(total_data, COLUMNS)

    def Excel(self, total_data, order):
        partidos_totales = []
        partido = []
        for i in total_data:
            local, vist = i[0]
            cuo_local, cuo_emp, cuo_vist = i[1]
            fecha, hora = i[2]
            local_emp, local_vist, vist_emp = i[3]
            si_anotan, no_anotan = i[4]
            datos_part = (local, vist, cuo_local, cuo_emp, cuo_vist, fecha, hora,
                          local_emp, local_vist, vist_emp, si_anotan, no_anotan)
            partido.append(datos_part)
        partidos_totales.extend(partido)

        df = pd.DataFrame(partidos_totales, columns=order)
        nuevos_nombre = {
            'vist':'visitante',
            'cuo_local':'cuota_local',
            'cuo_emp':'cuota_empate',
            'cuo_vist':'cuota_visitante',
            'local_emp':'local/empate',
            'local_vist':'local/visitante',
            'vist_emp':'visitante/empate'
        }
        df = df.rename(columns=nuevos_nombre)

        writer = pd.ExcelWriter('cuotas.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name='Cuotas_1', index=False)
        writer.close()

## pruebas

In [96]:
urls = ['https://apuestas.wplay.co/es/Ligue1']

obj = Create_Excel(urls)


analizando >>https://apuestas.wplay.co/es/Ligue1<<
